In [1]:
import os
import re
import requests
import pandas as pd
from io import StringIO, BytesIO

10000-20000: Antwerpen  
11000-12000: Admin Antwerpen  
12000-13000: Admin Mechelen  
13000-14000: Admin Turnhout  

21000-22000: Brussel  
22000-24000: Vlaams-Brabant  
22000-23000: Admin Halle-Vilvoorde  
23000-24000: Admin Leuven  

30000-40000: West-Vlaanderen  
40000-50000: Oost-Vlaanderen  
70000-80000: Limburg  

In [2]:
df = pd.read_csv("REFNIS_2019.csv", encoding='latin1', sep=";")
df.drop(["Code INS","Entités administratives","Langue"], axis=1, inplace=True)
df.drop([f"Unnamed: {idx}" for idx in range(6,15)], axis=1, inplace=True)

admin = df[df["Code NIS"].apply(lambda x: 10000<=x<20000) & df["Code NIS"].apply(lambda x: x%1000 is 0)]

drop = df[df["Code NIS"].apply(lambda x: 10000<=x<20000) & df["Code NIS"].apply(lambda x: x%1000 is 0)]
province = df[df["Code NIS"].apply(lambda x: 10000<=x<20000)]
province = province.sort_values(["Administratieve eenheden"]) # Sort by name
province[province["Administratieve eenheden"].apply(lambda x: x[0] == "A")] # Starts with letter
pass

In [4]:
#NIS = nis_lookup["Antwerpen"]["Duffel"]
#print(NIS)
#print(type(NIS))
#url = f"https://eservices.minfin.fgov.be/myminfin-rest/cadastral-plan/cadastralPlan/2020/12009/72"
#req = requests.get(url)

In [33]:
url = "https://statbel.fgov.be/sites/default/files/Over_Statbel_FR/Nomenclaturen/REFNIS_DEFINITIEF.csv"

In [35]:
refnis = pd.read_csv(StringIO(requests.get(url).content.decode("latin1")), sep=";")
refnis.drop(["Code INS","Entités administratives","Langue"], axis=1, inplace=True)

In [2]:
class NIS:
    def __init__(self,province:str="", place:str="", code:int=0):
        self.province = province
        self.place = place
        self.code = code
    
    @classmethod
    def from_place(cls, string:str=""):
        for province, places in cls.nis_lookup.items():
            for place, nis_code in places.items():
                if string.lower() == place.lower():
                    return NIS(province, place, nis_code)
        print("No entry found from given string")
        
    @classmethod
    def from_code(cls, code:int=0):
        for province, places in cls.nis_lookup.items():
            for place, nis_code in places.items():
                if code == nis_code:
                    return NIS(province, place, nis_code)
        print("No entry found from given code")
        
    @classmethod
    def collection(cls):
        for province, collection in cls.nis_lookup.items():
            for place, nis in collection.items():
                yield province, place, nis
            
    @classmethod
    def print_nis_lookup(cls):
        for province, collection in cls.nis_lookup.items():
            print(f"{province}\n" + "".join("-" for i in range(len(province))))
            for place, nis in collection.items():
                print(f"{nis} : {place}")
            print("\n")
    
    def initialize_dictionary():
        url = "https://statbel.fgov.be/sites/default/files/Over_Statbel_FR/Nomenclaturen/REFNIS_DEFINITIEF.csv"
        refnis = pd.read_csv(StringIO(requests.get(url).content.decode("latin1")), sep=";")
        refnis.drop(["Code INS","Entités administratives","Langue"], axis=1, inplace=True)
        
        result = {}
        province_nis_range = {
        "Antwerpen":       lambda x: 10000 <=x <20000,
        "Brussel":         lambda x: 21000 <=x <22000,
        "Vlaams-Brabant":  lambda x: 22000 <=x <24000,
        "West-Vlaanderen": lambda x: 30000 <=x <40000,
        "Oost-Vlaanderen": lambda x: 40000 <=x <50000,
        "Limburg":         lambda x: 70000 <=x <80000}        
        for province, nis_range in province_nis_range.items():
            dataframe = refnis[refnis["Code NIS"].apply(nis_range) 
                              &refnis["Code NIS"].apply(lambda x: x%1000 is not 0)]
            dataframe = dataframe.sort_values("Administratieve eenheden")
            result[province] = dict(zip(
                (item[1] for item in dataframe["Administratieve eenheden"].items()),
                (item[1] for item in dataframe["Code NIS"].items())))
        return result    
    nis_lookup = initialize_dictionary()
NIS.print_nis_lookup()

Antwerpen
---------
11001 : Aartselaar
11002 : Antwerpen
13001 : Arendonk
13002 : Baarle-Hertog
13003 : Balen
13004 : Beerse
12002 : Berlaar
11004 : Boechout
12005 : Bonheiden
11005 : Boom
12007 : Bornem
11007 : Borsbeek
11008 : Brasschaat
11009 : Brecht
13006 : Dessel
12009 : Duffel
11013 : Edegem
11016 : Essen
13008 : Geel
13010 : Grobbendonk
12014 : Heist-op-den-Berg
11018 : Hemiksem
13011 : Herentals
13012 : Herenthout
13013 : Herselt
13014 : Hoogstraten
11021 : Hove
13016 : Hulshout
11022 : Kalmthout
11023 : Kapellen
13017 : Kasterlee
11024 : Kontich
13053 : Laakdal
12021 : Lier
13019 : Lille
11025 : Lint
11057 : Malle
12025 : Mechelen
13021 : Meerhout
13023 : Merksplas
13025 : Mol
11029 : Mortsel
11030 : Niel
12026 : Nijlen
13029 : Olen
13031 : Oud-Turnhout
12029 : Putte
12030 : Puurs
11035 : Ranst
13035 : Ravels
13036 : Retie
13037 : Rijkevorsel
11037 : Rumst
11038 : Schelle
11039 : Schilde
11040 : Schoten
12034 : Sint-Amands
12035 : Sint-Katelijne-Waver
11044 : Stabroek
13040 :

In [5]:
from zipfile import ZipFile

def get_shapefiles_from_nis_code(data_sets:[], nis:NIS):
    url = f"https://eservices.minfin.fgov.be/myminfin-rest/cadastral-plan/cadastralPlan/2020/{nis.code}/72"
    print(requests.get(url).content)
    for data_set in data_sets:
        with ZipFile(BytesIO(requests.get(url).content)) as package:
            for content in package.namelist():
                if re.match(f".*({data_set}).*", content):
                    root_dir = f"./{nis.province}/"
                    directory = root_dir + f"{nis.place}/"
                    if not os.path.exists(root_dir): os.mkdir(root_dir)
                    if not os.path.exists(directory): os.mkdir(directory)
                    with open(f"{directory}{content}", "wb") as file:
                        file.write(package.read(content))

get_shapefiles_from_nis_code(["CaBu","ReBu","CaBl"], NIS.from_place("Puurs"))
#get_shapefiles_from_nis_code(["CaBu","ReBu","CaBl"], NIS)

#shape = gpd.read_file(f"./Test/Bli_EaZo.shp")

b''


BadZipFile: File is not a zip file